In [0]:
# Imports
import tensorflow as tf
from constants import *  # Import hyperparameters
import warnings

warnings.filterwarnings("ignore")  # Suppress warnings (use with caution)

# Ensure TensorFlow is installed (latest stable version recommended)
!pip install tensorflow


In [0]:
RUN_NUMBER = 2
tf.random.set_seed(RUN_NUMBER)

from hyperparameters import HYPERPARAMETERS as HP
BATCH_SIZE = HP["BATCH_SIZE"]


In [ ]:
# TensorBoard setup
log_dir = "./logs"
summary_writer = tf.summary.create_file_writer(log_dir)

with summary_writer.as_default():
    tf.summary.scalar("Train Loss", total_loss / TRAIN_LEN, step=epoch)
    tf.summary.scalar("Validation Loss", val_loss, step=epoch)


In [0]:
# Specify paths for saving results
CHECKPOINT_PATH = './checkpoints/'
RESULTS_PATH = f'./results/{ENCODER_MODEL}/{RUN_NUMBER}/'

# Ensure paths exist
os.makedirs(CHECKPOINT_PATH, exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

TRAIN_FROM_SCRATCH = True

# Read the data
train_en, train_de, test_en, test_de, val_en, val_de = readdata()

# Tokenize English dataset
tok_train_en, tok_val_en, tok_test_en, train_en_sen_len, val_en_sen_len, \
test_en_sen_len, en_dict_w2i, en_dict_i2w, en_max_words = tokenize(
    train_en, val_en, test_en, max_length=MAX_INPUT_SIZE)
del train_en, val_en, test_en

# Add PAD token to English vocabulary
en_dict_w2i.update({'<PAD>': 0})
en_dict_i2w.update({0: '<PAD>'})
en_vocab_size = np.amax(tok_train_en)

# Tokenize German dataset
tok_train_de, tok_val_de, tok_test_de, train_de_sen_len, val_de_sen_len, \
test_de_sen_len, de_dict_w2i, de_dict_i2w, de_max_words = tokenize(
    train_de, val_de, test_de, max_length=MAX_INPUT_SIZE)
del train_de, val_de, test_de

# Add PAD token to German vocabulary
de_dict_w2i.update({'<PAD>': 0})
de_dict_i2w.update({0: '<PAD>'})
de_vocab_size = np.amax(tok_train_de)

# Create GloVe embedding matrix
glove_path = './glove.6B.300d.txt'  # Update with the correct path
glove_embedding_matrix = create_embedding_indexmatrix(
    en_max_words, EMBEDDING_DIM, en_dict_i2w, glove_path)
if not os.path.exists(glove_path):
    raise FileNotFoundError(f"GloVe file not found at {glove_path}")


In [0]:
# Number of sentences and steps per epoch
BUFFER_SIZE = 10000  # Can also be len(tok_train_en) if dataset size is small
steps_per_epoch = len(tok_train_en) // BATCH_SIZE

# Create training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((tok_train_en, tok_train_de))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(
    BATCH_SIZE, padded_shapes=([None], [None])
)

# Create validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((tok_val_en, tok_val_de))
val_dataset = val_dataset.padded_batch(
    BATCH_SIZE, padded_shapes=([None], [None])
)

# Create test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((tok_test_en, tok_test_de))
test_dataset = test_dataset.padded_batch(
    BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=False
)

# Clean up unused objects to save memory
del tok_train_en, tok_train_de, tok_val_en, tok_val_de
gc.collect()


In [0]:
from Encoder import CNNEncoder
from Decoder import LSTMDecoder
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant

# Initialize embedding layer
embed = Embedding(
    input_dim=len(en_dict_w2i),
    output_dim=EMBEDDING_DIM,
    embeddings_initializer=Constant(glove_embedding_matrix),
    input_length=MAX_INPUT_SIZE,
    trainable=False
)

# Initialize encoder
encoder = CNNEncoder(
    batch_size=BATCH_SIZE,
    drop_out=DROP_OUT,
    embedding_dim=EMBEDDING_DIM,
    max_input_size=MAX_INPUT_SIZE,
    kernel_size=KERNEL_SIZE
)

# Initialize decoder
decoder = LSTMDecoder(
    batch_size=BATCH_SIZE, 
    drop_out=DROP_OUT, 
    r_drop_out=R_DROP_OUT,
    max_input_size=MAX_INPUT_SIZE, 
    embedding_dim=EMBEDDING_DIM,
    vocab_size=len(de_dict_w2i)
)

# Define optimizer
if USE_ANNEALING:
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=SGD_LEARNING_RATE,
        decay_steps=steps_per_epoch,
        decay_rate=0.96,
        staircase=True
    )
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
else:
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

# Define loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    """
    Compute sparse categorical cross-entropy loss with masking for padding tokens.

    Args:
        real: Tensor of ground truth labels.
        pred: Tensor of predicted logits.

    Returns:
        Masked loss value.
    """
    mask = tf.cast(tf.not_equal(real, 0), dtype=tf.float32)
    loss_ = loss_object(real, pred)
    loss_ *= mask
    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

In [0]:
# Ensure checkpoint directory exists
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# Initialize checkpoint
checkpoint_prefix = os.path.join(CHECKPOINT_PATH, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

# Load weights if not training from scratch
if not TRAIN_FROM_SCRATCH:
    latest_checkpoint = tf.train.latest_checkpoint(CHECKPOINT_PATH)
    if latest_checkpoint:
        checkpoint.restore(latest_checkpoint).expect_partial()
        print(f'Checkpoint restored from {latest_checkpoint}')
    else:
        print('No checkpoint found. Training from scratch.')


In [0]:
#Loss function used during training 

# Softmax function
def softmax(x):
    exp_x = tf.exp(x)
    return exp_x / tf.reduce_sum(exp_x, axis=-1, keepdims=True)

# Evaluate function
def evaluate(inp, targ, compute_perp=False):
    batch_loss = 0
    perp = 0

    # Compute embeddings
    inp_embed = embed(inp)
    mask = tf.cast(tf.not_equal(inp, 0), dtype=tf.float32)
    Mask = tf.expand_dims(mask, -1) * EMBEDDING_DIM

    H, h_1, c_1, h_2, c_2 = encoder(inp_embed, Mask)
    if h_1 is None:
        h_1 = decoder.initialize_hidden_state()
        c_1 = decoder.initialize_hidden_state()
        h_2 = decoder.initialize_hidden_state()
        c_2 = decoder.initialize_hidden_state()

    context = decoder.initialize_hidden_state()
    z_t = tf.cast(tf.expand_dims([de_dict_w2i['bos']] * BATCH_SIZE, 1), dtype=tf.float32)

    for t in range(1, MAX_INPUT_SIZE):
        predictions, h_1, c_1, h_2, c_2, context = decoder(z_t, h_1, c_1, h_2, c_2, H, context)
        if compute_perp:
            real = targ[:, t]
            prob = tf.nn.softmax(predictions, axis=-1)
            selected_probs = tf.gather_nd(prob, tf.expand_dims(real, axis=-1), batch_dims=1)
            log_probs = -tf.math.log(selected_probs)
            perp += tf.reduce_sum(log_probs * mask[:, t])

        batch_loss += loss_function(targ[:, t], predictions)
        z_t = tf.expand_dims(targ[:, t], 1)  # Using teacher forcing

    return batch_loss, tf.exp(perp / tf.reduce_sum(mask))

# Validate function
def validate_or_test(dataset, sent_len=None, compute_perp=False):
    total_loss, total_perp = 0, 0
    for inp, targ in dataset:
        batch_loss, batch_perp = evaluate(inp, targ, compute_perp)
        total_loss += batch_loss
        if compute_perp:
            total_perp += batch_perp

    avg_loss = total_loss / len(dataset)
    avg_perp = None
    if compute_perp and sent_len is not None:
        total_words = sum(sent_len) - 2 * len(dataset)
        avg_perp = np.power(2.0, -total_perp / total_words)

    return avg_loss, avg_perp




In [ ]:
# Initialize variables for early stopping
sec_last_loss, last_loss, lowest_val_loss = float('inf'), float('inf'), float('inf')
delta = 0.0  # Minimum change in validation loss to continue training
early_stop_count = 0
EARLY_STOP_THRESHOLD = 5  # Stop training if no improvement for 5 consecutive epochs

for epoch in range(EPOCHS):
    start = time.time()
    total_loss = 0

    # Training step
    for batch, (inp, targ) in enumerate(train_dataset.take(steps_per_epoch)):
        with tf.GradientTape() as tape:
            inp_embed = embed(inp)
            mask = tf.cast(tf.not_equal(inp, 0), dtype=tf.float32)
            Mask = tf.expand_dims(mask, -1) * EMBEDDING_DIM

            H, h_1, c_1, h_2, c_2 = encoder(inp_embed, Mask)
            context = decoder.initialize_hidden_state()
            z_t = tf.cast(tf.expand_dims([de_dict_w2i['bos']] * BATCH_SIZE, 1), dtype=tf.float32)

            batch_loss = 0
            for t in range(1, MAX_INPUT_SIZE):
                predictions, h_1, c_1, h_2, c_2, context = decoder(z_t, h_1, c_1, h_2, c_2, H, context)
                batch_loss += loss_function(targ[:, t], predictions)
                z_t = tf.expand_dims(targ[:, t], 1)  # Teacher forcing

        gradients = tape.gradient(batch_loss, encoder.trainable_variables + decoder.trainable_variables)
        if USE_GRADIENT_CLIPPING:
            gradients, _ = tf.clip_by_global_norm(gradients, 5.0)

        optimizer.apply_gradients(zip(gradients, encoder.trainable_variables + decoder.trainable_variables))
        total_loss += batch_loss

        if batch % 25 == 0:
            print(f"Batch {batch}: Loss = {batch_loss / BATCH_SIZE}")

    # Time taken for the epoch
    time_taken = time.time() - start

    # Validation step
    val_loss, val_perp = validate_or_test(
    dataset=val_dataset, 
    sent_len=val_en_sen_len, 
    compute_perp=True
    )

    # TensorBoard Logging
    with summary_writer.as_default():
        tf.summary.scalar("Train Loss", total_loss / TRAIN_LEN, step=epoch)
        tf.summary.scalar("Validation Loss", val_loss, step=epoch)
        if val_perp is not None:
            tf.summary.scalar("Validation Perplexity", val_perp, step=epoch)

    # Early stopping logic
    if val_loss < lowest_val_loss:
        checkpoint.save(file_prefix=checkpoint_prefix)
        lowest_val_loss = val_loss
        early_stop_count = 0
        print(f"Checkpoint saved at epoch {epoch + 1}")
    else:
        early_stop_count += 1
        print(f"No improvement for {early_stop_count} epochs")
        if early_stop_count >= EARLY_STOP_THRESHOLD:
            print("Early stopping triggered. Training terminated.")
            break  # Exit the training loop if no improvement for threshold epochs

    # Logging
    print(f"Epoch {epoch + 1}: Train Loss = {total_loss / TRAIN_LEN:.4f}, Val Loss = {val_loss:.4f}")
    print(f"Time taken for epoch {epoch + 1}: {time_taken:.2f} seconds")

# End of training

In [ ]:
def output_predictions(inp, targ, encoder, decoder, vocab_i2w):
    predictions = []
    inp_embed = embed(inp)
    mask = tf.cast(tf.not_equal(inp, 0), dtype=tf.float32)
    Mask = tf.expand_dims(mask, -1) * EMBEDDING_DIM

    H, h_1, c_1, h_2, c_2 = encoder(inp_embed, Mask)
    context = decoder.initialize_hidden_state()
    z_t = tf.cast(tf.expand_dims([vocab_i2w['<BOS>']] * inp.shape[0], 1), dtype=tf.float32)

    for t in range(MAX_INPUT_SIZE):
        pred, h_1, c_1, h_2, c_2, context = decoder(z_t, h_1, c_1, h_2, c_2, H, context)
        pred_word = tf.argmax(pred, axis=-1)
        predictions.append(pred_word)
        z_t = pred_word

    return predictions


In [0]:
# Validate on the test dataset
test_loss, test_perp = validate_or_test(
    dataset=test_dataset, 
    sent_len=test_de_sen_len, 
    compute_perp=True
)


# Print test results
print(f"Test Loss: {test_loss:.4f}")
if test_perp is not None:
    print(f"Test Perplexity: {test_perp:.4f}")
else:
    print("Test Perplexity: Not computed")
